In [3]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt

####################################
# Plot Psi(x,t), rho(x,t), V(x) and tilde{Psi}(k,t)
####################################
def plot_psi(zeitschritt,x,psi_x,rho_x,k,psi_k,V):
    fig, axs = plt.subplots(3)
    axs[1].plot(x, np.real(psi_x[zeitschritt,:]), "red", label='Re[$\psi(x,t)$]')
    axs[1].plot(x, np.imag(psi_x[zeitschritt,:]), "green", label='Im[$\psi(x,t)$]')

    axs[0].plot(x, rho_x[zeitschritt,:], "black", label='$|\psi(x,t)|^2$')
    axs[0].plot(x, V, "blue", label='$V(x)$')
    
    axs[2].plot(k, np.real(psi_k[zeitschritt,:]), "red", label='Re[$\~\psi(k,t)]$')
    axs[2].plot(k, np.imag(psi_k[zeitschritt,:]), "green", label='Im[$\~\psi(k,t)]$')

    axs[0].legend(loc='upper right')
    axs[1].legend(loc='upper right')
    axs[2].legend(loc='upper right')
    fig.set_size_inches(10, 12)
    axs[0].set_ylim([0, 1])
    axs[1].set_ylim([-1, 1])
    axs[2].set_ylim([-1, 1])
    axs[2].set_xlim([-10, 10])
    
    axs[0].set_xlim([xminplot, xmaxplot])
    axs[1].set_xlim([xminplot, xmaxplot])

    plt.rcParams['font.size'] = '20'

    plt.show()

####################################
# Initialisiering des x-grids (Ortsraum)
####################################
def init_xgrid(Nx,xmin,xmax):
    dx=(xmax-xmin)/(Nx-1)
    xgrid=np.zeros(Nx-1)
    for i in range(0,Nx-1):
        xgrid[i]=xmin+dx*i
    return xgrid

####################################
# Initialisiering des k-grids (Impulsraum)
####################################
def init_kgrid(Nx,xmin,xmax):
    dx=(xmax-xmin)/(Nx-1)
    dk = 2*np.pi/(xmax-xmin)
    kgrid=np.zeros(Nx-1)
    for i in range(0,Nx-1):
        kgrid[i]=dk*(i-Nx/2)
    return kgrid

####################################
# Initialisiering des t-grids (Zeit-Gitter)
####################################
def init_tgrid(Nt,tmin,tmax):
    dt=(tmax-tmin)/(Nt-1)
    tgrid=np.zeros(Nt)
    for i in range(0,Nt):
        tgrid[i]=tmin+dt*i
    return tgrid

####################################
# Initialisiering der Wellenfunktion im Ortsraum für alle Zeitschritte
####################################
def init_psi_x(t,x):
    psi=np.zeros((t.size,x.size),dtype=complex)
    return psi

####################################
# Initialisiering der Wellenfunktion im Impulsraum für alle Zeitschritte
####################################
def init_psi_k(t,k):
    psi=np.zeros((t.size,k.size),dtype=complex)
    return psi

####################################
# Initialisiering des Potentials
####################################
def init_V(x):
    V=np.zeros((x.size),dtype=complex)
    return V

####################################
# Initialisiering einer Gaußförmigen Potentialbarriere
####################################
def init_V_barrier(x,x0,V0,d):
    V=np.zeros((x.size),dtype=complex)
    #for i in range(0,x.size):
        #if (abs(x[i]-x0)<d):
    V=(V0*np.exp(-0.5*((x-x0)/d)**2))
    return V

####################################
# Berechne Transformationsmatrix der diskreten Fourier Transformation
####################################
def setup_dft_fast(x,k):
    dx=(x[1]-x[0])
    dk=(k[1]-k[0])
    dft=np.zeros((k.size,x.size),dtype=complex)
    for i in range(0,k.size):
        for j in range(0,x.size):
            dft[i,j]=dx/np.sqrt(2*np.pi)*np.exp(-1j*k[i]*x[j])
    return dft

####################################
# Berechne Transformationsmatrix der inversen diskreten Fourier Transformation
####################################
def setup_idft_fast(x,k):
    dx=(x[1]-x[0])
    dk=(k[1]-k[0])
    idft=np.zeros((x.size,k.size),dtype=complex)
    for j in range(0,x.size):
        for i in range(0,k.size):
            idft[j,i]=dk/np.sqrt(2*np.pi)*np.exp(1j*k[i]*x[j])
    return idft

####################################
# Diskrete Fourier Transformation (mittels Matrix-Vekor Multiplikation)
####################################
def dft_fast(psi_x,dft,x,k):
    psi_k=np.zeros(k.size,dtype=complex)
    psi_k=dft.dot(psi_x)
    return psi_k

####################################
# Inverse diskrete Fourier Transformation (mittels Matrix-Vekor Multiplikation)
####################################
def idft_fast(psi_k,idft,x,k):
    psi_x=np.zeros(x.size,dtype=complex)
    psi_x=idft.dot(psi_k)
    return psi_x

####################################
# Diskrete Fourier Transformation (mittels Loops)
####################################
def dft_slow(psi_x,x,k):
    dx=(x[1]-x[0])
    dk=(k[1]-k[0])
    psi_k=np.zeros(k.size,dtype=complex)
    for i in range(0,k.size):
        for j in range(0,x.size):
            psi_k[i]=psi_k[i]+dx/np.sqrt(2*np.pi)*np.exp(-1j*k[i]*x[j])*psi_x[j]
    return psi_k

####################################
# Inverse diskrete Fourier Transformation (mittels Loops)
####################################
def idft_slow(psi_k,x,k):
    dx=(x[1]-x[0])
    dk=(k[1]-k[0])
    psi_x=np.zeros(x.size,dtype=complex)
    for j in range(0,x.size):
        for i in range(0,k.size):
            psi_x[j]=psi_x[j]+dk/np.sqrt(2*np.pi)*np.exp(1j*k[i]*x[j])*psi_k[i]
    return psi_x

####################################
# Berechne Aufenthaltswahrscheinlichkeitsdichte
####################################
def calc_rho(psi):
    rho=np.abs(psi)
    return rho

####################################
# Zeitentwicklung der Wellenfunktion mittels SOP-Verfahren
####################################
def solve_td_se(hbar,m,t,x,k,psi_x,psi_k,V):

    #Berechnung der diskreten Fouriertransformationsmatrizen
    dft=setup_dft_fast(x,k)
    idft=setup_idft_fast(x,k)
    
    dt=t[1]-t[0] #Zeitschrittlänge
    
    #Berechnung des Propgators im Ortsraum (Potentialterm)
    x_propagate = np.exp(-1j*V/hbar*dt)
    
    #Berechnung des Propgators im Impulsraum (Kinetischer Energieterm)
    k_propagate_half = np.exp(-1j*hbar*k**2*dt/2./m)
    k_propagate = k_propagate_half*k_propagate_half
    
    #Transformation der Wellenfunktion vom Ortsraum in den Impulsraum
    psi_k[0,:]=dft_fast(psi_x[0,:],dft,x,k)
    #Propagation der Wellenfunktion im Impulsraum
    psi_tmp_k=psi_k[0,:]*k_propagate_half
    #Transformation der Wellenfunktion vom Impulsraum in den Ortsraum
    psi_tmp_x=idft_fast(psi_tmp_k,idft,x,k)
    
    for i in range(1,t.size):
        #Ausgabe des momentanen Zeitschritts
        print("Zeitschritt "+str(i)+"/"+str(t.size),end='\r')
        
        #Propagation der Wellenfunktion im Ortsraum (Potentialterm)
        psi_tmp_x=psi_tmp_x*x_propagate
        #Transformation der Wellenfunktion vom Ortsraum in den Impulsraum
        psi_tmp_k=dft_fast(psi_tmp_x,dft,x,k)
        #Propagation der Wellenfunktion im Impulsraum (kin. Energieterm)
        psi_tmp_k=psi_tmp_k*k_propagate
        psi_k[i,:]=psi_tmp_k
        #Inverse Transformation der Wellenfunktion vom Impulsraum in den Ortsraum
        psi_tmp_x=idft_fast(psi_tmp_k,idft,x,k)
        psi_x[i,:]=psi_tmp_x


####################################
# Initialisierung eines Gaußförmigen Wellenpaketes im Ortsraum
####################################
def gauss_packet_x(x,x0,k0,a):
    return (1./np.sqrt(a*np.sqrt(np.pi))
            *np.exp(-0.5*((x-x0)/a)**2+1j*x*k0))

####################################
# Initialisierung eines Gaußförmigen Wellenpaketes im Impulsraum
####################################
def gauss_packet_k(k,k0,d):
    return (np.exp(-((k-k0)*d)**2))

####################################################################
# ------- Ab hier werden Paramter der Simulation definiert ------- #
####################################################################

####################################
# Initialisierung der Variablen
####################################
hbar=1
m=1
    
####################################
# Initialisierung des Zeit-Gitters
####################################
tmin=0
tmax=200
Nt=400

t=init_tgrid(Nt,tmin,tmax)

####################################
# Initialisierung des 1D-Gitters im Orts- und Impulsraum
####################################
xmin=-100
xmax=100
Nx=2**10

xminplot=-100
xmaxplot=100

x=init_xgrid(Nx,xmin,xmax)
k=init_kgrid(Nx,xmin,xmax)

####################################
# Initialisierung des Potentials im Ortsraum
####################################
V=init_V(x)
b0=40 #Zentrum der Gaußförmigen Barriere
b=2 #Breite der Gaußförmigen Barriere
V0=15 #Höhe der Gaußförmigen Barriere
V=init_V_barrier(x,b0,V0,b)

####################################
# Initialisierung der Wellenfunktion im Orts- und Impulsraum
####################################

psi_x=init_psi_x(t,x)
psi_k=init_psi_k(t,k)

d=2   #Breite des Gaußförmigen Wellenpaketes im Impulsraum
k0=2  #Zentrum des Gaußförmigen Wellenpaketes im Impulsraum

psi_k[0,:]=gauss_packet_k(k,k0,d)
psi_x[0,:]=idft_slow(psi_k[0,:],x,k)

solve_td_se(hbar,m,t,x,k,psi_x,psi_k,V)

rho_x=calc_rho(psi_x)

interact(plot_psi, zeitschritt = widgets.IntSlider(value=0,
                                               min=0,
                                               max=Nt-1,
                                               step=1),
        x=fixed(x),psi_x=fixed(psi_x),rho_x=fixed(rho_x),
        k=fixed(k),psi_k=fixed(psi_k),V=fixed(V))



interactive(children=(IntSlider(value=0, description='zeitschritt', max=399), Output()), _dom_classes=('widget…

<function __main__.plot_psi(zeitschritt, x, psi_x, rho_x, k, psi_k, V)>